### Reddit Fake News Detector Project

This project is another personal project of mine.

In [1]:
'''
#pip install praw 

NOTE: We install praw specifically using pip here because Conda version is
outdated. A seperate Conda environment was created to avoid package management issues.
'''

'\n#pip install praw \n\nNOTE: We install praw specifically using pip here because Conda version is\noutdated. A seperate Conda environment was created to avoid package management issues.\n'

In [2]:
import random #import these packages for the following praw program in the next cell
import socket
import sys

import time #import time to assign a counter to the incoming subreddit stream
import praw as praw #import praw package for streaming content from Reddit

In [3]:
'''
Code taken from praw documentation to initiate program to obtain refresh token for Reddit API
authorisation. This is necessary to avoid using personal username and password for
authentication.
'''
def receive_connection():
    """Wait for and then return a connected socket..

    Opens a TCP connection on port 8080, and waits for a single client.

    """
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(("localhost", 8080))
    server.listen(1)
    client = server.accept()[0]
    server.close()
    return client


def send_message(client, message):
    """Send message to client and close the connection."""
    print(message)
    client.send(f"HTTP/1.1 200 OK\r\n\r\n{message}".encode("utf-8"))
    client.close()


def main():
    """Provide the program's entry point when directly executed."""
    print(
        "Go here while logged into the account you want to create a token for: "
        "https://www.reddit.com/prefs/apps/"
    )
    print(
        "Click the create an app button. Put something in the name field and select the"
        " script radio button."
    )
    print("Put http://localhost:8080 in the redirect uri field and click create app")
    client_id = input(
        "Enter the client ID, it's the line just under Personal use script at the top: "
    )
    client_secret = input("Enter the client secret, it's the line next to secret: ")
    commaScopes = input(
        "Now enter a comma separated list of scopes, or all for all tokens: "
    )

    if commaScopes.lower() == "all":
        scopes = ["*"]
    else:
        scopes = commaScopes.strip().split(",")

    reddit = praw.Reddit(
        client_id=client_id.strip(),
        client_secret=client_secret.strip(),
        redirect_uri="http://localhost:8080",
        user_agent="praw_refresh_token_example",
    )
    state = str(random.randint(0, 65000))
    url = reddit.auth.url(scopes, state, "permanent")
    print(f"Now open this url in your browser: {url}")
    sys.stdout.flush()

    client = receive_connection()
    data = client.recv(1024).decode("utf-8")
    param_tokens = data.split(" ", 2)[1].split("?", 1)[1].split("&")
    params = {
        key: value for (key, value) in [token.split("=") for token in param_tokens]
    }

    if state != params["state"]:
        send_message(
            client,
            f"State mismatch. Expected: {state} Received: {params['state']}",
        )
        return 1
    elif "error" in params:
        send_message(client, params["error"])
        return 1

    refresh_token = reddit.auth.authorize(params["code"])
    send_message(client, f"Refresh token: {refresh_token}")
    return 0


if __name__ == "__main__":
    sys.exit(main())
    

Go here while logged into the account you want to create a token for: https://www.reddit.com/prefs/apps/
Click the create an app button. Put something in the name field and select the script radio button.
Put http://localhost:8080 in the redirect uri field and click create app


Enter the client ID, it's the line just under Personal use script at the top:  93b74cYiSei-YQ
Enter the client secret, it's the line next to secret:  uUqBxVmcwVAqsYZ-fov-3v4srA3QuA
Now enter a comma separated list of scopes, or all for all tokens:  all


Now open this url in your browser: https://www.reddit.com/api/v1/authorize?client_id=93b74cYiSei-YQ&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&scope=%2A&state=34507
Refresh token: 690585682570-9V4hg5yFby2qE2K218_vsat86791vQ


SystemExit: 0

/home/luca/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
reddit = praw.Reddit(client_id = "93b74cYiSei-YQ", 
                                   client_secret = "uUqBxVmcwVAqsYZ-fov-3v4srA3QuA",
                                  user_agent = "FakeNewsDetector by u/VertexBanshee",
                                  refresh_token = "690585682570-9V4hg5yFby2qE2K218_vsat86791vQ"
                                  )

#Authentication credentials used above to get access to Reddit API
#Insert generated refresh token from the previous cell into the empty parameter

In [ ]:
print(reddit.user.me()) #check that the user login worked correctly

In [5]:
subreddit = reddit.subreddit("ukpolitics") #Set the subreddit of choice to stream Reddit post submissions. 

In [18]:
x = subreddit.new(limit=100)

In [19]:
def streamer():

    for submission in x:
        print(submission.title)
        

In [20]:
print(streamer())

COVID-19: Schools will not reopen for all pupils after February half-term, PM confirms
Brexit Britain set for '50 years' of non-stop negotiations with EU to improve deal
TUV's Allister issues unionist rallying call to 'disrupt' Brexit protocol and 'prevent Union's destruction'
Covid-19: PM to speak as tougher quarantine control expected
Climate Change Doesn’t Have Time for Broken Two-Party Politics
Covid: Police deal with 'incident' near Wrexham vaccine plant
HS2 protesters dig 100ft tunnel under London park
Here are five ways the government could have avoided 100,000 Covid deaths
EDF announces delay, price rise for UK nuclear plant
Contactless limit could rise to £100
Discontent growing over Northern Ireland protocol, police warn
Plan B for independence is a road to nowhere
Britain’s First Sea Lord prepares to embark on year in spotlight
AstraZeneca cancels steering board meeting with EU
EU and BioNTech/Pfizer clash over reduced vaccine deliveries
UK aid cuts of up to 70% a 'gut punch